In [32]:
#read data

import pandas as pd
train_file_path = "project_2_train/project_2_train/dara 1_train.csv"
train_data = pd.read_csv(train_file_path)
train_data.head()

,example_id,text,aspect_term,term_location,class
0,2333_1,Obviously one of the most important features o...,human interface,69--84,0
1,1805_1,Good for every day computing and web browsing.,every day computing,9--28,1
2,2782_2,while the keyboard itself is alright[comma] th...,mouse command buttons,115--136,-1
3,1385_0,Again[comma] the same problem[comma] the right...,right speaker,29--42,-1
4,1423_0,My problem was with DELL Customer Service.,DELL Customer Service,20--41,-1


In [33]:
aspect_term = train_data[' aspect_term'].value_counts(dropna=False)
print(aspect_term)

use                               51
screen                            50
price                             50
battery life                      46
battery                           42
keyboard                          38
programs                          35
warranty                          31
features                          29
software                          29
hard drive                        28
size                              22
performance                       21
quality                           21
Windows                           20
speed                             20
memory                            17
applications                      16
runs                              14
Vista                             14
system                            13
charge                            13
motherboard                       13
graphics                          13
gaming                            12
program                           12
works                             12
d

In [34]:
class_value = train_data[' class'].value_counts(dropna=False)
print(class_value)

 1    939
-1    828
 0    436
Name:  class, dtype: int64


In [35]:
X = train_data.drop([" class","example_id"," text"],axis=1)
X.head()

,aspect_term,term_location
0,human interface,69--84
1,every day computing,9--28
2,mouse command buttons,115--136
3,right speaker,29--42
4,DELL Customer Service,20--41


In [36]:
X.shape

(2203, 2)

In [37]:
## target data
Y = train_data[" class"]
Y.head()

0    0
1    1
2   -1
3   -1
4   -1
Name:  class, dtype: int64

In [38]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_bag = count_vect.fit_transform(train_data[" text"])
X_train_bag.shape

(2203, 3246)

In [39]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bag)
X_train_tfidf.shape
X_tfidf = pd.DataFrame(X_train_tfidf.toarray())

In [40]:

df2 = pd.merge(X,X_tfidf, left_index=True, right_index=True)

In [41]:
df2.head()

,aspect_term,term_location,0,1,2,3,4,5,6,7,...,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245
0,human interface,69--84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,every day computing,9--28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,mouse command buttons,115--136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,right speaker,29--42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DELL Customer Service,20--41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
##train validation data split

from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(df2, Y,random_state = 0)

In [42]:
one_hot_encoded_training_predictors = pd.get_dummies(df2)
#one_hot_encoded_val_predictors = pd.get_dummies(val_X)
#final_train, final_val = one_hot_encoded_training_predictors.align(one_hot_encoded_val_predictors,
#                                                                    join='left', 
#                                                                    axis=1)

In [43]:
from sklearn.preprocessing import Imputer
my_imputer = Imputer()
imputed_X_train = my_imputer.fit_transform(one_hot_encoded_training_predictors)
#imputed_X_val = my_imputer.transform(final_val)
#print(imputed_X_train.shape)
print(imputed_X_val.shape)

(551, 5085)


In [44]:
#Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(imputed_X_train, Y)

In [15]:
#Naive Bayes Prediction
import numpy as np
from sklearn.metrics import accuracy_score
predicted = clf.predict(imputed_X_val)
accuracy_score(val_Y, predicted)

0.647912885662432

In [47]:
#Naive Bayes using cross validation and
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
clf = MultinomialNB()
predicted = cross_val_predict(clf, imputed_X_train, Y, cv=10)
conf_mat_naive = confusion_matrix(Y,predicted,labels=[-1,0,1])
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

For positive class
Precision
0.7011494252873564
Recall
0.842720510095643
F1 Score
0.7654440154440154
For negative class
Precision
0.6692223439211391
Recall
0.736144578313253
F1 Score
0.7010900745840505
For neutral class
Precision
0.6792452830188679
Recall
0.2465753424657534
F1 Score
0.36180904522613067
None
(array([0.66922234, 0.67924528, 0.70114943]), array([0.73792271, 0.24770642, 0.84451544]), array([0.70189546, 0.36302521, 0.76618357]), array([828, 436, 939]))


In [48]:

accuracy_score(Y, predicted)

0.6863368134362233

In [19]:
#Random Forest train
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier()
forest_model.fit(imputed_X_train,train_Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [20]:
#Random Forest predict
forest_pred = forest_model.predict(imputed_X_val)
accuracy_score(val_Y, forest_pred)

0.6751361161524501

In [50]:
#Random Forest using cross validation and confusion matrix
forest_model = RandomForestClassifier()
predicted = cross_val_predict(forest_model, imputed_X_train, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
calclulateF1(conf_mat_forest)
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

For positive class
Precision
0.7540453074433657
Recall
0.742826780021254
F1 Score
0.7483940042826552
For negative class
Precision
0.6663254861821903
Recall
0.7843373493975904
F1 Score
0.7205312672938572
For neutral class
Precision
0.6187290969899666
Recall
0.4223744292237443
F1 Score
0.5020352781546811
For positive class
Precision
0.7011494252873564
Recall
0.842720510095643
F1 Score
0.7654440154440154
For negative class
Precision
0.6692223439211391
Recall
0.736144578313253
F1 Score
0.7010900745840505
For neutral class
Precision
0.6792452830188679
Recall
0.2465753424657534
F1 Score
0.36180904522613067
None
(array([0.66632549, 0.6187291 , 0.75404531]), array([0.78623188, 0.42431193, 0.74440895]), array([0.72132964, 0.50340136, 0.74919614]), array([828, 436, 939]))


In [52]:
accuracy_score(Y, predicted)

0.6967771221062188

In [17]:
def calclulateF1(conf_matrix):
    print("For positive class")
    tp_positive = conf_matrix[2,2]
    fn_positive = sum(conf_matrix[2,:], 2)-conf_matrix[2, 2]
    fp_positive = sum(conf_matrix[:,2])-conf_matrix[2, 2]
    print("Precision")
    p_positive = tp_positive/(tp_positive+fp_positive)
    print(p_positive)
    print("Recall")
    r_positive =tp_positive/(tp_positive+fn_positive)
    print(r_positive)
    print("F1 Score")
    f1_positive = (2*p_positive*r_positive)/(p_positive+r_positive)
    print(f1_positive)
    print("For negative class")
    tp_negative = conf_matrix[0,0]
    fn_negative = sum(conf_matrix[0,:], 2)-conf_matrix[0, 0]
    fp_negative = sum(conf_matrix[:,0])-conf_matrix[0, 0]
    print("Precision")
    p_negative = tp_negative/(tp_negative+fp_negative)
    print(p_negative)
    print("Recall")
    r_negative =tp_negative/(tp_negative+fn_negative)
    print(r_negative)
    print("F1 Score")
    f1_negative = (2*p_negative*r_negative)/(p_negative+r_negative)
    print(f1_negative)
    print("For neutral class")
    tp_neutral = conf_matrix[1,1]
    fn_neutral = sum(conf_matrix[1,:], 2)-conf_matrix[1, 1]
    fp_neutral = sum(conf_matrix[:,1])-conf_matrix[1, 1]
    print("Precision")
    p_neutral = tp_neutral/(tp_neutral+fp_neutral)
    print(p_neutral)
    print("Recall")
    r_neutral =tp_neutral/(tp_neutral+fn_neutral)
    print(r_neutral)
    print("F1 Score")
    f1_neutral = (2*p_neutral*r_neutral)/(p_neutral+r_neutral)
    print(f1_neutral)

In [23]:
from sklearn.ensemble import BaggingClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

clf1 = BaggingClassifier()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = MLPClassifier()

vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
vote_model.fit(imputed_X_train,train_Y)
vote_pred = vote_model.predict(imputed_X_val)
accuracy_score(val_Y, vote_pred)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6987295825771325

In [53]:
#Random Forest using cross validation and confusion matrix
vote_model = VotingClassifier(estimators=[('bag', clf1), ('rf', clf2), ('mlp', clf3)], voting='hard')
predicted = cross_val_predict(vote_model, imputed_X_train, Y, cv=10)
conf_mat_forest = confusion_matrix(Y,predicted,labels=[-1,0,1])
calclulateF1(conf_mat_forest)
print(calclulateF1(conf_mat_naive))
print(precision_recall_fscore_support(Y,predicted))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

For positive class
Precision
0.7839305103148752
Recall
0.767268862911796
F1 Score
0.7755102040816326
For negative class
Precision
0.6962283384301733
Recall
0.8228915662650602
F1 Score
0.7542794036443954
For neutral class
Precision
0.6578073089700996
Recall
0.4520547945205479
F1 Score
0.5358592692828146
For positive class
Precision
0.7011494252873564
Recall
0.842720510095643
F1 Score
0.7654440154440154
For negative class
Precision
0.6692223439211391
Recall
0.736144578313253
F1 Score
0.7010900745840505
For neutral class
Precision
0.6792452830188679
Recall
0.2465753424657534
F1 Score
0.36180904522613067
None
(array([0.69622834, 0.65780731, 0.78393051]), array([0.82487923, 0.45412844, 0.76890309]), array([0.75511332, 0.53731343, 0.77634409]), array([828, 436, 939]))


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [55]:
accuracy_score(Y, predicted)

0.7276441216522923